# Tic Tac Toe Keychain

SEE: https://medium.com/@thepgb/programming-puzzle-tic-tac-toe-keychain-e247cc8c4ec7

In [3]:
def board2str(board):
  return '\n'.join(''.join(_) for _ in board)

board = [['.'] * 3, ['.'] * 3, ['.'] * 3]
print(board2str(board))

board[1][1] = 'x'
board[2][2] = 'o'
print('\n' + board2str(board))

...
...
...

...
.x.
..o


In [4]:
def is_final_state(board):
  if sum(board[r][c] != '.' for r in range(3) for c in range(3)) == 9:
    # All spaces filled.
    return True

  for win in ('x', 'o'):
    # Check wins along a row.
    for r in range(3):
      if all(piece == win for piece in board[r]):
        return True

    # Check wins along a column.
    for c in range(3):
      if all(board[r][c] == win for r in range(3)):
        return True

    # Check diagonals.
    if board[0][0] == win and board[1][1] == win and board[2][2] == win:
      return True
    if board[0][2] == win and board[1][1] == win and board[2][0] == win:
      return True

  return False

for board in [
    [
      list('...'),
      list('.x.'),
      list('..o'),
    ],
    [
      list('..x'),
      list('..x'),
      list('..x'),
    ],
    [
      list('o.x'),
      list('.ox'),
      list('..o'),
    ],
    [
      list('xoo'),
      list('oxx'),
      list('xxo'),
    ]
]:
    print('\n' + board2str(board))
    print('is_final_state:', is_final_state(board))


...
.x.
..o
is_final_state: False

..x
..x
..x
is_final_state: True

o.x
.ox
..o
is_final_state: True

xoo
oxx
xxo
is_final_state: True


In [5]:
def next_states(board):
  next_piece = 'x' if sum(board[r][c] != '.' for r in range(3) for c in range(3)) % 2 == 0 else 'o'
  for r in range(3):
    for c in range(3):
      if board[r][c] == '.':
        next_board = [row.copy() for row in board]
        next_board[r][c] = next_piece
        yield next_board


for next_board in next_states([
  list('o.x'),
  list('.ox'),
  list('...'),
]):
  print('\n' + board2str(next_board))


oxx
.ox
...

o.x
xox
...

o.x
.ox
x..

o.x
.ox
.x.

o.x
.ox
..x


In [6]:
def dfs(board, valids):
  valids.append([row.copy() for row in board])
  if not is_final_state(board):
    for next_board in next_states(board):
      dfs(next_board, valids)

import time

valids = []
start_time = time.time()
dfs([['.'] * 3 for _ in range(3)], valids)
stop_time = time.time()
print('Num valids:', len(valids), 'took:', stop_time - start_time, 's')

Num valids: 549946 took: 5.915194034576416 s


In [7]:
def dfs2(board, valids):
  board_str = board2str(board)
  if board_str not in valids:
    valids.add(board_str)
    if not is_final_state(board):
      for next_board in next_states(board):
        dfs2(next_board, valids)

import time

valids = set()
start_time = time.time()
dfs2([['.'] * 3 for _ in range(3)], valids)
stop_time = time.time()
print('Num valids:', len(valids), 'took:', stop_time - start_time, 's')

Num valids: 5478 took: 0.08420372009277344 s


In [8]:
def dfs3(cells, valids):
  if len(cells) == 9:
    board = [cells[:3], cells[3:6], cells[6:]]
    if is_valid(board):
      valids.append([row.copy() for row in board])
  else:
    dfs3(cells + ['.'], valids)
    dfs3(cells + ['x'], valids)
    dfs3(cells + ['o'], valids)


def is_valid(board):
  return True

valids_exhaustive = []
dfs3([], valids_exhaustive)
print('Valids:', len(valids_exhaustive))

Valids: 19683


In [9]:
def is_valid(board):
  num_xs = sum(sum(_ == 'x' for _ in row) for row in board)
  num_os = sum(sum(_ == 'o' for _ in row) for row in board)

  if (num_xs - num_os) not in (0, 1):
    return False

  return True

valids_xo = []
dfs3([], valids_xo)
print('X-O count valids:', len(valids_xo))


X-O count valids: 6046


In [10]:
def check_wins(board, win):
  win_positions = set()

  # Check wins along a row.
  for r in range(3):
    if all(piece == win for piece in board[r]):
      win_positions.update([(r, c) for c in range(3)])

  # Check wins along a column.
  for c in range(3):
    if all(board[r][c] == win for r in range(3)):
      win_positions.update([(r, c) for r in range(3)])

  # Check diagonals.
  if board[0][0] == win and board[1][1] == win and board[2][2] == win:
    win_positions.update([(0, 0), (1, 1), (2, 2)])
  if board[0][2] == win and board[1][1] == win and board[2][0] == win:
    win_positions.update([(0, 2), (1, 1), (2, 0)])

  return win_positions


def is_valid(board):
  num_xs = sum(sum(_ == 'x' for _ in row) for row in board)
  num_os = sum(sum(_ == 'o' for _ in row) for row in board)

  if (num_xs - num_os) not in (0, 1):
    return False

  x_wins = check_wins(board, 'x')
  o_wins = check_wins(board, 'o')
  if x_wins and o_wins:
    return False

  return True

valids_xo_double_win = []
dfs3([], valids_xo_double_win)
print('X-O count, no double wins valids:', len(valids_xo_double_win))
print(set(map(board2str, valids_xo_double_win)) - valids)

X-O count, no double wins valids: 5890
{'xxo\nxoo\nxo.', 'xxx\nxoo\no.o', '.xo\n.xo\nox.', '.xx\nooo\nx.x', '.xo\noxo\n.x.', 'ooo\nxx.\nxx.', 'oxo\nxxx\noo.', 'oox\noxo\nxx.', 'oxo\nox.\nxxo', 'o.x\noxo\nx..', 'xxx\noxo\noo.', 'xx.\nxoo\nxoo', 'xoo\nx..\nxo.', 'o.x\n.ox\nxxo', 'ox.\n.xo\n.xo', 'xxx\n.o.\no.o', 'o.o\noox\nxxx', 'xoo\nox.\n..x', 'o.o\noxo\nxxx', 'xoo\nxxo\nxo.', 'oxx\no..\noxx', '.ox\n.x.\nxoo', 'oox\noox\nx.x', '.ox\nxxo\nxoo', 'oo.\n..o\nxxx', '.ox\nxox\nxo.', 'xoo\nox.\noxx', 'xoo\nxx.\nxoo', 'oox\noxo\nx.x', 'x.o\n.ox\noxx', 'oxx\nxox\nxoo', 'oo.\nxxx\no..', 'oox\no.x\nxox', 'ooo\n.xx\nx.x', 'xxo\n..o\nxxo', 'ox.\no.x\noxx', 'o.x\noxo\nxxo', 'oo.\nxxx\n..o', 'x..\noxo\no.x', '.xo\nxxo\nx.o', 'oxx\n.xo\nxoo', 'xox\n.xo\noox', 'x.x\nxx.\nooo', 'oxo\n.oo\nxxx', 'xoo\nxo.\nx..', 'oxo\no.o\nxxx', '..x\noox\no.x', 'o.o\nxxx\noxo', 'xxx\n.oo\no..', 'xoo\noxx\n.ox', 'ox.\n.xo\nox.', 'oxx\no.x\nox.', 'x..\nx.o\nxoo', '.xo\nx.o\nxxo', 'xo.\n.xo\no.x', 'xox\nxox\n.o.', '.xx\n.x

In [11]:
def is_valid(board):
  num_xs = sum(sum(_ == 'x' for _ in row) for row in board)
  num_os = sum(sum(_ == 'o' for _ in row) for row in board)

  if (num_xs - num_os) not in (0, 1):
    return False

  x_wins = check_wins(board, 'x')
  o_wins = check_wins(board, 'o')
  if x_wins and o_wins:
    return False
  if x_wins and num_xs != num_os + 1:
    return False
  if o_wins and num_xs != num_os:
    return False

  return True

valids_xo_fixed_win = []
dfs3([], valids_xo_fixed_win)
print('X-O count, fixed win valids:', len(valids_xo_fixed_win))

X-O count, fixed win valids: 5478


In [12]:
def pos2board(pos):
  c = lambda c: 'x' if c == 1 else ('o' if c == 2 else '.')
  return [
    [c(pos[0]), c(pos[1]), c(pos[2])],
    [c(pos[3]), c(pos[4]), c(pos[5])],
    [c(pos[6]), c(pos[7]), c(pos[8])],
  ]

print(board2str(pos2board([0, 2, 1, 0, 1, 0, 0, 1, 2])))

.ox
.x.
.xo


In [14]:
valid_counting = set()
for i in range(3 ** 9):
  pos = []
  v = i
  for j in range(9):
    pos.append(i % 3)
    i = i // 3

  board = pos2board(pos)
  if is_valid(board):
    valid_counting.add(board2str(board))

print('Counting valids:', len(valid_counting))

Counting valids: 5478
